In [ ]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
import numpy as np
import swifter
import matplotlib.pyplot as plt
import rasterio
import datetime
import os, shutil

### Settings 

In [ ]:
# Slumps
INFERENCE_DIR = Path(r'Q:\p_aicore_pf\initze\processed\inference\RTS_6Regions_V01_UnetPlusPlus_resnet34_FocalLoss_sh6_50_bs100_2021-12-04_22-27-53')
# Pingo
#INFERENCE_DIR = Path(r'Q:\p_aicore_pf\initze\processed\inference\pingo_UnetPP_v1_2021-12-12_09-56-50')

OUTPUT_DIR = Path(r'C:\Users\initze\OneDrive\100_AI-CORE\16_inference_statistics')
out_file = OUTPUT_DIR / f'{INFERENCE_DIR.stem}_merged_datasets.shp'

In [ ]:
print(out_file)

### create filelist

In [ ]:
flist = list(INFERENCE_DIR.glob('*'))

In [ ]:
def get_vector(f):
    gdf = gpd.read_file(f).to_crs(epsg=4326)
    gdf['id_local'] = gdf.index
    gdf['dataset'] = f.stem
    gdf['model'] = f.parts[-2]
    gdf[['scene', 'tile_id', 'date', 'sensor']] = f.stem.split('_')
    return gdf

#### Load files and add to list 

In [ ]:
ds_list = []
for f in flist[:]:
    try:
        ds_list.append(get_vector(f))
    except:
        print(f'Error on {f.stem}')

#### Merge all GDF to one 

In [ ]:
rdf = gpd.GeoDataFrame( pd.concat( ds_list, ignore_index=True) )

#### Set projection (got lost during merge with pandas) 

In [ ]:
rdf = rdf.set_crs(epsg=4326)

#### Write to file

In [ ]:
rdf.to_file(out_file)